![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

# Use pretrained `match_datetime` Pipeline

### Spark `2.4` and Spark NLP `2.0.1`

* DocumentAssembler
* SentenceDetector
* Tokenizer
* DateMatcher `yyyy/MM/dd`

In [2]:
import sys
sys.path.append('../../')

#Spark ML and SQL
from pyspark.ml import Pipeline, PipelineModel
from pyspark.sql.functions import array_contains
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
#Spark NLP
from sparknlp.annotator import *
from sparknlp.common import RegexRule
from sparknlp.base import DocumentAssembler, Finisher

### Let's create a Spark Session for our app

In [3]:
import sparknlp
from sparknlp.pretrained import PretrainedPipeline

spark = sparknlp.start()

In [4]:
spark.version

'2.4.0'

In [5]:
pipeline = PretrainedPipeline('match_datetime')

In [7]:
result = pipeline.annotate("Let's meet on 20th of February.")

In [8]:
result['date']

['2019/02/20']

In [9]:
result=lp.annotate("I would like to come over and see you in 01/02/2019.")

In [10]:
result['date']

['2019/01/02']